In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# StageLoader: shipments_Stage Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_shipments` (
         `bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`shipment_id`
        ,`store_id`
        ,`customer_id`
        ,`delivery_address`
        ,`shipment_status`
    )
    SELECT
         {bg_loadtimestamp} AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`shipment_id` AS `shipment_id`
        ,`bg_source`.`store_id` AS `store_id`
        ,`bg_source`.`customer_id` AS `customer_id`
        ,`bg_source`.`delivery_address` AS `delivery_address`
        ,`bg_source`.`shipment_status` AS `shipment_status`
    FROM `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_shipments_source` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['reloadtarget_{marc_databricks_initiative#stage#database_name}_{marc_databricks_initiative#stage#schema_name}_stg_st_shipments'] = operation_metrics

except Exception as e:
    info('STG_ST_shipments_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
